In [1]:
import requests
import json
from stock_valuation.income_statement import IncomeStatement
from stock_valuation.balance_sheet import BalanceSheet
from stock_valuation.cash_flow_statement import CashFlowStatement


ALPHA_VANTAGE_API_KEY = "WGD32KDX07IA26F3"
stock_symbol = "UBER"

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key

is_resp = requests.get(f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={stock_symbol}&apikey={ALPHA_VANTAGE_API_KEY}").json()
bs_resp = requests.get(f"https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={stock_symbol}&apikey={ALPHA_VANTAGE_API_KEY}").json()
# cf_resp = requests.get(f"https://www.alphavantage.co/query?function=CASH_FLOW&symbol={stock_symbol}&apikey={ALPHA_VANTAGE_API_KEY}").json()

In [2]:
income_statements = [IncomeStatement.from_alpha_vantage(data) for data in is_resp["annualReports"]][::-1]
balance_sheets = [BalanceSheet.from_alpha_vantage(data) for data in bs_resp["annualReports"]][::-1][-5:]

In [3]:
income_statements

[IncomeStatement(revenue=13000000000, cost_of_revenue=15299000000, operating_expenses=13029000000, selling_general_and_administrative=7925000000, research_and_development=4836000000, deprecation_and_amortization=166000000, interest_expense=115000000, provision_for_taxes=45000000, consolidated_net_income=-8512000000, net_income=-8506000000),
 IncomeStatement(revenue=11139000000, cost_of_revenue=12523000000, operating_expenses=9650000000, selling_general_and_administrative=6249000000, research_and_development=2205000000, deprecation_and_amortization=354000000, interest_expense=458000000, provision_for_taxes=-192000000, consolidated_net_income=-6788000000, net_income=-6768000000),
 IncomeStatement(revenue=17455000000, cost_of_revenue=17228000000, operating_expenses=10040000000, selling_general_and_administrative=7105000000, research_and_development=2054000000, deprecation_and_amortization=656000000, interest_expense=483000000, provision_for_taxes=-492000000, consolidated_net_income=-57000

In [4]:
balance_sheets

[BalanceSheet(inventory=99000000, accounts_receivables=1642000000, other_current_assets=300000000, total_current_assets=13925000000, property_plant_and_equipment=1731000000, goodwill=167000000, total_assets=31761000000, short_term_debt=196000000, accounts_payable=272000000, other_current_liabilities=298000000, total_current_liabilities=5639000000, long_term_debt=5791000000, total_liabilities=16578000000, total_shareholder_equity=14190000000),
 BalanceSheet(inventory=767000000, accounts_receivables=1537000000, other_current_assets=344000000, total_current_assets=9882000000, property_plant_and_equipment=1814000000, goodwill=6109000000, total_assets=33252000000, short_term_debt=352000000, accounts_payable=235000000, other_current_liabilities=1796000000, total_current_liabilities=6865000000, long_term_debt=7914000000, total_liabilities=19498000000, total_shareholder_equity=12266000000),
 BalanceSheet(inventory=1643000000, accounts_receivables=2992000000, other_current_assets=442000000, tot

In [5]:
is_ratios_list = []

for i in range(1, len(income_statements)):
    prev_is, prev_bs = income_statements[i - 1], balance_sheets[i - 1]
    is_ratios_list.append(income_statements[i].get_ratios(prev_is, prev_bs))

In [6]:
is_ratios_list

[IncomeStatementRatios(revenue_growth=-0.1431538461538462, gross_margin=1.124248137175689, operating_margin=0.866325522937427, sga_to_revenue=0.5610018852679773, research_and_development_to_revenue=0.19795313762456235, deprecation_and_amortization_to_ppe=0.2045060658578856, interest_expense_to_total_debt=0.07649908134290964, taxes_to_pretax_profit=0.027507163323782235, minority_interest_to_consolidated_net_income=0.0029463759575721863),
 IncomeStatementRatios(revenue_growth=0.5670167878624652, gross_margin=0.9869951303351475, operating_margin=0.5751933543397307, sga_to_revenue=0.4070466914924091, research_and_development_to_revenue=0.11767401890575767, deprecation_and_amortization_to_ppe=0.36163175303197354, interest_expense_to_total_debt=0.05843213162351803, taxes_to_pretax_profit=0.4632768361581921, minority_interest_to_consolidated_net_income=0.12982456140350876),
 IncomeStatementRatios(revenue_growth=0.8262389000286452, gross_margin=0.89312043165919, operating_margin=0.367757317187

In [7]:
bs_ratios_list = []

for i in range(1, len(balance_sheets)):
    bs_ratios_list.append(balance_sheets[i].get_ratios(income_statements[i]))

In [8]:
bs_ratios_list

[BalanceSheetRatios(accounts_receivables_to_revenue=0.13798366101086273, inventory_to_revenue=0.06885716850704732, ppe_to_revenue=0.1628512433791184, accounts_payables_to_cost_of_revenue=0.02109704641350211),
 BalanceSheetRatios(accounts_receivables_to_revenue=0.17141220280721856, inventory_to_revenue=0.09412775708965912, ppe_to_revenue=0.10615869378401604, accounts_payables_to_cost_of_revenue=0.04926955027212833),
 BalanceSheetRatios(accounts_receivables_to_revenue=0.10945195595570474, inventory_to_revenue=0.02133199485522477, ppe_to_revenue=0.06531354895379113, accounts_payables_to_cost_of_revenue=0.022837782727358282),
 BalanceSheetRatios(accounts_receivables_to_revenue=0.1105388803948392, inventory_to_revenue=0.0, ppe_to_revenue=0.055604731632735174, accounts_payables_to_cost_of_revenue=0.021190418711944423)]

## Build forecast for IS/BS ratios

In [9]:
from dataclasses import fields
from stock_valuation.income_statement_ratios import IncomeStatementRatios
from stock_valuation.balance_sheet_ratios import BalanceSheetRatios


def forecast_income_statement_ratios(is_ratios_list, extrapolation_funcs):
    forecasted_fields = {}
    for f in fields(IncomeStatementRatios):
        extrapolation_func = extrapolation_funcs[f.name]
        field_values = [getattr(is_ratios, f.name) for is_ratios in is_ratios_list]
        forecasted_fields[f.name] = extrapolation_func(field_values)
    
    return IncomeStatementRatios(**forecasted_fields)

def forecast_balance_sheet_ratios(bs_ratios_list, extrapolation_funcs):
    forecasted_fields = {}
    for f in fields(BalanceSheetRatios):
        extrapolation_func = extrapolation_funcs[f.name]
        field_values = [getattr(bs_ratios, f.name) for bs_ratios in bs_ratios_list]
        forecasted_fields[f.name] = extrapolation_func(field_values)
    
    return BalanceSheetRatios(**forecasted_fields)
        
def last_value(values):
    return values[-1]

In [12]:
is_extrpl_funcs = {
    "revenue_growth": last_value,
    "gross_margin": last_value,
    "operating_margin": last_value,
    "sga_to_revenue": last_value,
    "research_and_development_to_revenue": last_value,
    "deprecation_and_amortization_to_ppe": last_value,
    "interest_expense_to_total_debt": last_value,
    "taxes_to_pretax_profit": last_value,
    "minority_interest_to_consolidated_net_income": last_value,
}
bs_extrpl_funcs = {
    "accounts_receivables_to_revenue": last_value,
    "inventory_to_revenue": last_value,
    "ppe_to_revenue": last_value,
    "accounts_payables_to_cost_of_revenue": last_value,
}

forecasted_is_ratios = forecast_income_statement_ratios(is_ratios_list, is_extrpl_funcs)
forecasted_bs_ratios = forecast_balance_sheet_ratios(bs_ratios_list, bs_extrpl_funcs)

In [13]:
forecasted_is_ratios

IncomeStatementRatios(revenue_growth=0.16952661793769797, gross_margin=0.8059064939245192, operating_margin=0.29087202596496875, sga_to_revenue=0.18878248974008208, research_and_development_to_revenue=0.08486896810707867, deprecation_and_amortization_to_ppe=0.2641690682036503, interest_expense_to_total_debt=0.06422483766233766, taxes_to_pretax_profit=0.0899113550021106, minority_interest_to_consolidated_net_income=0.12476808905380334)

In [14]:
forecasted_bs_ratios

BalanceSheetRatios(accounts_receivables_to_revenue=0.1105388803948392, inventory_to_revenue=0.0, ppe_to_revenue=0.055604731632735174, accounts_payables_to_cost_of_revenue=0.021190418711944423)

## Forecast Income Statement / Balance Sheet from Ratios

In [18]:
def forecast_is_from_ratios(prev_is: IncomeStatement, prev_bs: BalanceSheet, is_ratios: IncomeStatementRatios):
    revenue = prev_is.revenue * (1 + is_ratios.revenue_growth)
    operating_expenses = revenue * (1 - is_ratios.operating_margin)
    operating_income = revenue - operating_expenses  # EBIT
    interest_expense = prev_bs.total_debt * is_ratios.interest_expense_to_total_debt
    pretax_profit = operating_income - interest_expense
    provision_for_taxes = pretax_profit * is_ratios.taxes_to_pretax_profit
    consolidated_income = pretax_profit - provision_for_taxes
    
    return IncomeStatement(
            revenue=revenue,
            cost_of_revenue=revenue * (1 - is_ratios.gross_margin),
            operating_expenses=operating_expenses,
            selling_general_and_administrative=revenue * is_ratios.sga_to_revenue,
            research_and_development=revenue * is_ratios.research_and_development_to_revenue,
        
            deprecation_and_amortization=prev_bs.property_plant_and_equipment * is_ratios.deprecation_and_amortization_to_ppe,
            interest_expense=interest_expense,
            provision_for_taxes=provision_for_taxes,
            consolidated_net_income=consolidated_income,
            net_income=consolidated_income * (1 - is_ratios.minority_interest_to_consolidated_net_income),
    )

In [19]:
forecasted_is = forecast_is_from_ratios(income_statements[-1], balance_sheets[-1], forecasted_is_ratios)
forecasted_is

IncomeStatement(revenue=43601121843.33532, cost_of_revenue=8462694607.397183, operating_expenses=30918775198.418922, selling_general_and_administrative=8231128337.045519, research_and_development=3700382219.154876, deprecation_and_amortization=547622478.3861672, interest_expense=646037642.0454545, provision_for_taxes=1082200851.6722293, consolidated_net_income=10954108151.198713, net_income=9587385009.88496)

In [ ]:
def forecast_bs_from_ratios(curr_is: IncomeStatement, prev_bs: BalanceSheet, bs_ratios: BalanceSheetRatios):
    return BalanceSheet(
            # cash= TODO: after cash flow st only
            inventory=curr_is.revenue * bs_ratios.inventory_to_revenue,
            accounts_receivables=curr_is.revenue * bs_ratios.accounts_receivables_to_revenue,
            # short_term_investments=prev_bs.short_term_investments,
            other_current_assets=prev_bs.other_current_assets,
#             total_current_assets: int after cash

#             # long-term assets
            property_plant_and_equipment=curr_is.revenue * bs_ratios.ppe_to_revenue,
            goodwill=prev_bs.goodwill,
            other_long_term_assets=prev_bs.other_long_term_assets,

            # total_assets: int

#             # current (short-term) liabilites
            short_term_debt=curr_is.revenue * bs_ratios.short_term_debt_to_revenue,
            accounts_payable=curr_is.cost_of_revenue * bs_ratios.accounts_payable_to_cost_of_revenue
            other_current_liabilities=prev_bs.other_current_liabilities
#             total_current_liabilities: int

#             # long-term liabilites
#             long_term_debt: int

#             total_liabilities: int

#             # equity
#             total_shareholder_equity: int
    )